In [ ]:
import heapq

# define a priority queue class
class PriorityQueue:
    def __init__(self):
        self.elements = []

    # check if the priority queue is empty
    def empty(self):
        return len(self.elements) == 0

    # add an item with a given priority to the priority queue
    def put(self, item, priority):
        heapq.heappush(self.elements, (priority, item))

    # get the item with the highest priority from the priority queue
    def get(self):
        return heapq.heappop(self.elements)[1]

# define a grid class to represent the grid layout and distances
class Grid:
    def __init__(self, layout, distances):
        self.layout = layout
        self.distances = distances

    # get valid neighboring nodes for a given node
    def neighbors(self, node):
        x, y = node
        possible_neighbors = [(x - 1, y), (x + 1, y), (x, y - 1), (x, y + 1)]
        valid_neighbors = [(i, j) for i, j in possible_neighbors if 0 <= i < len(self.layout) and 0 <= j < len(self.layout[0])]
        return valid_neighbors

    # just toprint the grid with a path marked with 'X'
    def print_grid_with_path(self, path):
        grid_representation = [["." for _ in range(len(self.layout[0]))] for _ in range(len(self.layout))]

        for x, y in path:
            grid_representation[y][x] = "X"  # mark path with 'X'

        # print the table
        for row in grid_representation:
            print(" ".join(row))

# define a heuristic function for estimating the distance from 'a' to 'b'
def heuristic(a, b):
  return abs(a[0] - b[0]) + abs(a[1] - b[1])

# implement the A* search algorithm
def a_star_search(grid, start, goal):
    # create a priority queue for managing nodes to explore
    frontier = PriorityQueue()
    # initialize the priority queue with the starting node and a priority of 0
    frontier.put(start, 0)

    # create dictionaries to keep track of the paths and costs
    came_from = {}        # dictionary to store the path from the start to the current node
    cost_so_far = {}      # dictionary to store the cost from the start to the current node

    # initialize the dictionaries with the starting node
    came_from[start] = None
    cost_so_far[start] = 0

    # continue the search until the priority queue is empty
    while not frontier.empty():
        # get the node with the lowest total cost from the priority queue
        current = frontier.get()

        # if the current node is the goal, exit the loop
        if current == goal:
            break

        # explore the neighboring nodes of the current node
        for next_node in grid.neighbors(current):
            # calculate the new cost to reach the neighbor node from the current node
            new_cost = cost_so_far[current] + grid.distances.get((current, next_node), float('inf'))

            # check if the neighbor node has not been visited or if the new cost is lower than the previously recorded cost
            if next_node not in cost_so_far or new_cost < cost_so_far[next_node]:
                # update the cost and priority for the neighbor node
                cost_so_far[next_node] = new_cost
                priority = new_cost + heuristic(goal, next_node)

                # add the neighbor node to the priority queue for further exploration
                frontier.put(next_node, priority)

                # record the current node as the parent of the neighbor node
                came_from[next_node] = current

    # if the goal node is not in the 'came_from' dictionary, no valid path was found
    if goal not in came_from:
        return None

    # reconstruct the path from the goal node to the start node
    current = goal
    path = []
    while current is not None:
        path.append(current)
        current = came_from[current]

    # reverse the path to get it in the correct order
    path.reverse()

    # return the final path from the start to the goal
    return path

# define the layout of the grid and calculate distances between points
layout = [
    [(0, 0), (1, 0), (2, 0)],
    [(0, 1), (1, 1), (2, 1)],
    [(0, 2), (1, 2), (2, 2)]
]

distances = {}

# loop through the matrix to calculate distances
for x1, col1 in enumerate(layout):
    for y1, point1 in col1:
        for x2, col2 in enumerate(layout):
            for y2, point2 in col2:
                # calculate Manhattan distance between point1 and point2
                distance = abs(x1 - x2) + abs(y1 - y2)
                distances[(point1, point2)] = distance

# create a Grid instance with the layout and distances
grid = Grid(layout, distances)

# mapping the layout to coordinates
classroom_mapping = {
    "faculty1": (0, 0),
    "c1": (1, 0),
    "faculty2": (2, 0),
    "c2": (0, 1),
    "c3": (1, 1),
    "c4": (2, 1),
    "c5": (0, 2),
    "c6": (1, 2),
    "exit": (2, 2),
}

# user input for starting and goal classrooms
from tabulate import tabulate

data = [
    ["faculty1", "c1", "faculty2"],
    ["c2", "c3", "c4"],
    ["c5", "c6", "exit"]
]

# Print the table
table = tabulate(data, tablefmt="grid")
print(table)

start_classroom = input("Enter your starting node: ")
goal_classroom = input("Enter your goal node: ")

# check if the input classrooms are valid
if start_classroom in classroom_mapping and goal_classroom in classroom_mapping:
    start_location = classroom_mapping[start_classroom]
    goal_location = classroom_mapping[goal_classroom]

    # find the shortest path using A* search
    path = a_star_search(grid, start_location, goal_location)

    # if a path is found, print it and mark it on the grid
    if path:
        print("Shortest path:")
        grid.print_grid_with_path(path)
        for node in path:
            if isinstance(node, tuple):
                print(f"({node[0]}, {node[1]})")
            else:
                print(node)
    else:
        print("No path found.")
else:
    print("Invalid starting or goal classroom.")


+----------+----+----------+
| faculty1 | c1 | faculty2 |
+----------+----+----------+
| c2       | c3 | c4       |
+----------+----+----------+
| c5       | c6 | exit     |
+----------+----+----------+
Enter your starting node: c5
Enter your goal node: c4
Shortest path:
. . .
X X X
X . .
(0, 2)
(0, 1)
(1, 1)
(2, 1)
